In [1]:
import sklearn
import sktime
from tsai.basics import *

import os
import sys

import hydra
import numpy as np
import torch
import torch.nn.functional as F
import wandb  # API key: 2efbdd2316576b45689ac6f5c6e957865ad0b1ba
from omegaconf import DictConfig
from termcolor import cprint
from torchmetrics import Accuracy
from tqdm import tqdm

from src.datasets import ThingsMEGDataset
from src.models import BasicConvClassifier, miniROCKETClassifier
from src.utils import set_seed

## Pytorch implementation (any # samples)
from tsai.models.MINIROCKET_Pytorch import *
from tsai.models.utils import *

In [2]:
loader_args = {"batch_size": 128, "num_workers": 4}

train_set = ThingsMEGDataset("train", data_dir="data")
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, **loader_args)
val_set = ThingsMEGDataset("val", data_dir="data")
val_loader = torch.utils.data.DataLoader(val_set, shuffle=False, **loader_args)
test_set = ThingsMEGDataset("test", data_dir="data")
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, **loader_args)

In [3]:
print(train_set.num_classes)

1854


In [3]:
model = BasicConvClassifier(
    train_set.num_classes, train_set.seq_len, train_set.num_channels
).to("cuda:0")
model

BasicConvClassifier(
  (blocks): Sequential(
    (0): ConvBlock(
      (conv0): Conv1d(271, 128, kernel_size=(3,), stride=(1,), padding=same)
      (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=same)
      (batchnorm0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (batchnorm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): ConvBlock(
      (conv0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=same)
      (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=same)
      (batchnorm0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (batchnorm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (head): Sequential(
    (0): AdaptiveAvgPool1d(output_size=1)
    (1): Rearrange('b d 1 -> 

In [4]:
model = MiniRocket(
    c_in=train_set.num_channels,
    seq_len=train_set.seq_len,
    num_features=10_000,
    max_dilations_per_kernel=32,
    c_out=train_set.num_classes,
).to("cuda:0")
model

MiniRocket(
  (backbone): MiniRocketFeatures()
  (head): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): BatchNorm1d(9996, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=9996, out_features=1854, bias=True)
  )
)

In [5]:
from tsai.models.TCN import *

model = TCN(
        c_in=train_set.num_channels,
        c_out=train_set.num_classes,
        ks=7,
        layers=[25] * 3,
        conv_dropout=0.2,
        fc_dropout=0.2,
    ).to("cuda:0")

model

TCN(
  (tcn): Sequential(
    (0): TemporalBlock(
      (conv1): Conv1d(271, 25, kernel_size=(7,), stride=(1,), padding=(6,))
      (chomp1): Chomp1d()
      (relu1): ReLU()
      (dropout1): Dropout(p=0.2, inplace=False)
      (conv2): Conv1d(25, 25, kernel_size=(7,), stride=(1,), padding=(6,))
      (chomp2): Chomp1d()
      (relu2): ReLU()
      (dropout2): Dropout(p=0.2, inplace=False)
      (net): Sequential(
        (0): Conv1d(271, 25, kernel_size=(7,), stride=(1,), padding=(6,))
        (1): Chomp1d()
        (2): ReLU()
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv1d(25, 25, kernel_size=(7,), stride=(1,), padding=(6,))
        (5): Chomp1d()
        (6): ReLU()
        (7): Dropout(p=0.2, inplace=False)
      )
      (downsample): Conv1d(271, 25, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
    )
    (1): TemporalBlock(
      (conv1): Conv1d(25, 25, kernel_size=(7,), stride=(1,), padding=(12,), dilation=(2,))
      (chomp1): Chomp1d()
      (relu1): ReLU(

In [7]:
from tsai.models.RNN import *

model = LSTM(
        c_in=train_set.num_channels,
        c_out=train_set.num_classes,
        hidden_size=512,
        n_layers=2,
        rnn_dropout=0.2,
        fc_dropout=0.2,
        bidirectional=True,
    ).to("cuda:0")

model

LSTM(
  (rnn): LSTM(271, 512, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=1024, out_features=1854, bias=True)
)

In [9]:
print(train_loader.dataset[0][0].shape)

torch.Size([271, 281])
